In [1]:
#Question1
'''
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
    '''
#Run the above command to run the elaticsearch containers

#Then run the following command to obtain the result.
'''curl localhost:9200'''

'curl localhost:9200'

In [2]:
#Question2-3
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
from elasticsearch import Elasticsearch

In [4]:
es_client = Elasticsearch('http://localhost:9200') 

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
#Execute this only when creating the index for the first time
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
from tqdm.auto import tqdm

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [10]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    scores = []
    
    for hit in response['hits']['hits']:
        scores.append(hit['_score'])
    
    return scores

In [11]:
query="How do I execute a command in a running docker container?"

In [12]:
elastic_search(query)

[84.050095, 75.54128, 72.08518, 51.04628, 49.938507]

In [13]:
#Question4
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    questions = []
    
    for hit in response['hits']['hits']:
        questions.append(hit['_source']['question'])
    
    return questions

In [14]:
elastic_search(query)

['How do I debug a docker container?',
 'How do I copy files from my local machine to docker container?',
 'How do I copy files from a different folder into docker container’s working directory?',
 'The input device is not a TTY when running docker in interactive mode (Running Docker on Windows in GitBash)',
 'How to fix error after running the Docker run command']

In [15]:
#Question5
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    questions = []
    
    for hit in response['hits']['hits']:
        questions.append(hit['_source'])
    
    return questions

In [16]:
search_results=elastic_search(query)

In [24]:
#Question5
def build_prompt(query, search_results):
    context_template = """
Q: {question}
A: {text}
""".strip()
    
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context=context+context_template.format(question=doc['question'], text=doc['text'])+'\n\n'
    
    prompt = prompt_template.format(question=query, context=context).strip()
    print(len(prompt))
    return prompt

In [27]:
prompt=build_prompt(query,search_results)

1463


In [28]:
#Question6
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [29]:
# Encode the text
encoded_text = encoding.encode(prompt)
